# Tools

In [34]:
import numpy as np
from matplotlib.ticker import FormatStrFormatter, PercentFormatter


#割合(x,y,z)[%]をXY座標に変換 convert rate(x,y,z)[%] into XY-coordinate
def transform_xy(x, y, z=None):
    if z is None:
        z = 100 - x - y
    h = np.sqrt(3) * 0.5
    x_ = (z / 100) + (x / 100) / 2
    y_ = (x / 100) * h
    
    return x_, y_

def transform_xy_halfsize(x, y, z=None):
    if z is None:
        z = 100 - x - y
    h = np.sqrt(3) * 0.5
    x_ = (z / 100) * 2 + (x / 100) * 2 / 2 - 0.5
    y_ = (x / 100) * 2 * h - h
    
    xy = np.stack([x_, y_])
    idx = (xy >= 0).all(axis=0)
    xy = xy[..., idx]
    
    return xy[0], xy[1], idx


#plot Triangle
def plot_tri(x,y,z,d, label_x,label_y,label_z, *additional_points, show_cbar=True, cbarlabel=None, percentage=False, reduce_ticks=False, ax=None, half_size=False, cbar_kw={}, **kwargs):
    import matplotlib.pyplot as plt

    
    if not ax:
        fig, ax = plt.subplots(dpi=150)
    else:
        fig = ax.get_figure()
        
    l = len(d)
    ax.set_aspect('equal', 'datalim')
    ax.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False)
    ax.tick_params(bottom=False, left=False, right=False, top=False)
    ax.spines['bottom'].set_visible(False)
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    h = np.sqrt(3.0)*0.5

    #外周-三角
    ax.plot([0.0, 1.0],[0.0, 0.0], 'k-', lw=2)
    ax.plot([0.0, 0.5],[0.0, h], 'k-', lw=2)
    ax.plot([1.0, 0.5],[0.0, h], 'k-', lw=2)


    #内側目盛
    for i in range(1,10):
        ax.plot([i/20.0, 1.0-i/20.0],[h*i/10.0, h*i/10.0], color='gray', lw=0.5,ls="-")
        ax.plot([1.0-i/20.0-0.025, 1.0-i/20.0],[h*i/10.0, h*i/10.0], color='black', lw=2,ls="-")

        ax.plot([i/20.0, i/10.0],[h*i/10.0, 0.0], color='gray', lw=0.5)
        ax.plot([i/20.0, i/20+0.05/4],[h*i/10.0, h*i/10-h/10*0.25], color='black', lw=2)

        ax.plot([0.5+i/20.0, i/10.0],[h*(1.0-i/10.0), 0.0], color='gray', lw=0.5)
        ax.plot([i/10.0+0.025/2, i/10.0],[h/40, 0.0], color='black', lw=2)



    #頂点のラベル
    ax.text(0.75+0.25/2+0.01, h/2, label_x, fontsize='xx-large', rotation=0)#x
    ax.text(0.25/2-0.13, h/2, label_y, fontsize='xx-large')#y
    ax.text(0.5-0.05, -0.17, label_z, fontsize='xx-large', rotation=0)#z

    #軸ラベル
    step = 2 if reduce_ticks else 1
    interval = 5 if half_size else 10
    shift = 50 if half_size else 0
    for i in range(0,11, step):
        r = i*interval
        ax.text((10-i)/20.0-0.01, h*(10-i)/10.0, '%d' % r, fontsize='medium', rotation=0,horizontalalignment='right') #x
        ax.text(0.5+(10-i)/20.0+0.01, h*(1.0-(10-i)/10.0), '%d' % (r + shift), fontsize='medium',horizontalalignment='left')#y
        ax.text(i/10.0-0.03, -0.07, '%d' % r, fontsize='medium', rotation=0) #z
        
    if half_size:
        X, Y, idx = transform_xy_halfsize(x,y,z)
        d = d[idx]
    else:
        X, Y = transform_xy(x, y, z)

    sca = ax.scatter(X, Y, c=d, s=5, alpha=0.8, **kwargs)
    
    for fn in additional_points:
        fn(ax)

    # Create colorbar
    if show_cbar:
        cbar = ax.figure.colorbar(sca, ax=ax, **cbar_kw)
        if percentage:
            cbar.ax.yaxis.set_major_formatter(PercentFormatter(1))
        if cbarlabel is not None:
            cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom", fontsize='x-large')

    fig.tight_layout()

In [1]:
from itertools import product

class ConfusionMatrixDisplay:
    """Confusion Matrix visualization.
    It is recommend to use :func:`~sklearn.metrics.plot_confusion_matrix` to
    create a :class:`ConfusionMatrixDisplay`. All parameters are stored as
    attributes.
    Read more in the :ref:`User Guide <visualizations>`.
    Parameters
    ----------
    confusion_matrix : ndarray of shape (n_classes, n_classes)
        Confusion matrix.
    display_labels : ndarray of shape (n_classes,)
        Display labels for plot.
    Attributes
    ----------
    im_ : matplotlib AxesImage
        Image representing the confusion matrix.
    text_ : ndarray of shape (n_classes, n_classes), dtype=matplotlib Text, \
            or None
        Array of matplotlib axes. `None` if `include_values` is false.
    ax_ : matplotlib Axes
        Axes with confusion matrix.
    figure_ : matplotlib Figure
        Figure containing the confusion matrix.
    """
    def __init__(self, confusion_matrix, display_labels):
        self.confusion_matrix = confusion_matrix
        self.display_labels = display_labels

    def plot(self, *, include_values=True, cmap='viridis',
             xticks_rotation='horizontal', values_format=None, ax=None, anno_fontsize=14, tick_fontsize=14, label_fontsize=20):
        """Plot visualization.
        Parameters
        ----------
        include_values : bool, default=True
            Includes values in confusion matrix.
        cmap : str or matplotlib Colormap, default='viridis'
            Colormap recognized by matplotlib.
        xticks_rotation : {'vertical', 'horizontal'} or float, \
                         default='vertical'
            Rotation of xtick labels.
        values_format : str, default=None
            Format specification for values in confusion matrix. If `None`,
            the format specification is '.2f' for a normalized matrix, and
            'd' for a unnormalized matrix.
        ax : matplotlib axes, default=None
            Axes object to plot on. If `None`, a new figure and axes is
            created.
        Returns
        -------
        display : :class:`~sklearn.metrics.ConfusionMatrixDisplay`
        """
        import matplotlib.pyplot as plt

        if ax is None:
            fig, ax = plt.subplots()
        else:
            fig = ax.figure

        cm = self.confusion_matrix
        n_classes = cm.shape[0]
        self.im_ = ax.imshow(cm, interpolation='nearest', cmap=cmap)
        self.text_ = None

        cmap_min, cmap_max = self.im_.cmap(0), self.im_.cmap(256)

        if include_values:
            self.text_ = np.empty_like(cm, dtype=object)
            if values_format is None:
                values_format = '.2g'

            # print text with appropriate color depending on background
            thresh = (cm.max() - cm.min()) / 2.
            for i, j in product(range(n_classes), range(n_classes)):
                color = cmap_max if cm[i, j] < thresh else cmap_min
                self.text_[i, j] = ax.text(j, i,
                                           format(cm[i, j], values_format),
                                           ha="center", va="center",
                                           color=color, fontsize=anno_fontsize)

        cbar = fig.colorbar(self.im_, ax=ax)
        cbar.ax.tick_params(labelsize=anno_fontsize) 
        ax.set(xticks=np.arange(n_classes),
               yticks=np.arange(n_classes),
               xticklabels=self.display_labels,
               yticklabels=self.display_labels,
               ylabel="True label",
               xlabel="Predicted label")

        ax.set_ylim((n_classes - 0.5, -0.5))
        plt.setp(ax.get_xticklabels(), rotation=xticks_rotation)

        ax.xaxis.label.set_size(label_fontsize)
        ax.yaxis.label.set_size(label_fontsize)
        ax.tick_params(labelsize=tick_fontsize)
        
        self.figure_ = fig
        self.ax_ = ax
        return self


In [2]:
def make_virtual_compounds(*elements, interval=0.5, pen_size=10, **element_ratio):
    dim = len(elements)
    
    # for list type
    if dim != 0:
        # elements and element_ratio are mutually exclusive
        if len(element_ratio) != 0:
            raise ValueError('elements and element_ratio are mutually exclusive')

        # sort elements in the alphabetical order
        elements = tuple(sorted(tuple(set(elements))))
        
        # generate mesh
        grid = np.arange(0, 100, interval)
        mesh = np.array(np.meshgrid(*([grid] * dim))).T.reshape(-1, dim)

    # for dict type
    else:
        if len(element_ratio) == 0:
            return None

        # sort elements in the alphabetical order
        dim = len(element_ratio)
        elements, ratios = zip(*[(k, element_ratio[k]) for k in sorted(element_ratio.keys())])
        
        # generate mesh
        s = [np.arange(max(r - pen_size, 0), min(r + pen_size, 100), interval) for r in ratios]
        mesh = np.array(np.meshgrid(*s)).T.reshape(-1, dim)
    
    # select the only ratios have sum equal to 100
    mesh = mesh[mesh.sum(1) == 100]

    # assign element
    comps = [{e:f for e, f in zip(elements, frac_compositon) } for frac_compositon in mesh]
    # save as pandas dataframe
    return pd.DataFrame({'composition': comps, 'elements': [elements] * mesh.shape[0]})

In [9]:
import numpy as np
import pandas as pd
from typing import Union, List


from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
from sklearn.utils.class_weight import compute_sample_weight

def classification_metrics(
        y_true: Union[np.ndarray, pd.DataFrame, pd.Series],
        y_pred: Union[np.ndarray, pd.Series],
        *,
        average: Union[None, List[str]] = ['weighted', 'micro', 'macro'],
        labels = None,
) -> dict:
    """
    Calculate most common classification scores.
    See Also: https://scikit-learn.org/stable/modules/model_evaluation.html
    
    Parameters
    ----------
    y_true
        True results.
    y_pred
        Predicted results.
        
    Returns
    -------
    OrderedDict
        An :class:`collections.OrderedDict` contains classification scores.
        These scores will be calculated: ``accuracy``, ``f1``, ``precision``, ``recall``,
        ``macro_f1``, ``macro_precision``, and ``macro_recall``
    """
    if average is not None and len(average) == 0:
        raise ValueError('need average')
    
    if len(y_true.shape) != 1:
        y_true = np.argmax(y_true, 1)
    if len(y_pred.shape) != 1:
        y_pred = np.argmax(y_pred, 1)

    mask = ~np.isnan(y_pred)
    y_true = y_true[mask]
    y_pred = y_pred[mask]
    
    ret = dict(
        accuracy = accuracy_score(y_true, y_pred)
    )
    
    if average is None:
        ret.update(
            f1 = f1_score(y_true, y_pred, average=None, labels=labels),
            precision = precision_score(y_true, y_pred, average=None, labels=labels),
            recall = recall_score(y_true, y_pred, average=None, labels=labels),
        )
        
        return ret
    
    if 'weighted' in average:
        ret.update(
            f1 = f1_score(y_true, y_pred, average='weighted', labels=labels),
            precision = precision_score(y_true, y_pred, average='weighted', labels=labels),
            recall = recall_score(y_true, y_pred, average='weighted', labels=labels),
        )
        
    if 'micro' in average:
        ret.update(
            micro_f1 = f1_score(y_true, y_pred, average='micro', labels=labels),
            micro_precision = precision_score(y_true, y_pred, average='micro', labels=labels),
            micro_recall = recall_score(y_true, y_pred, average='micro', labels=labels),
        )
    
    if 'macro' in average:
        ret.update(
            macro_f1 = f1_score(y_true, y_pred, average='macro', labels=labels),
            macro_precision = precision_score(y_true, y_pred, average='macro', labels=labels),
            macro_recall = recall_score(y_true, y_pred, average='macro', labels=labels),
        )
    
    return ret
